In [2]:
import os
import numpy as np
import pandas as pd
from sklearn import neighbors, metrics 
from sklearn.neural_network import MLPClassifier,MLPRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import KBinsDiscretizer  

LOOKING INTO THE DATASETS:

In [3]:
#Importing the dataset
data_GTAV = pd.read_csv('FPS_grandTheftAuto5.csv') 
data_Fortnite = pd.read_csv('FPS_fortnite.csv') 
data_FarCry5 = pd.read_csv('FPS_farCry5.csv') 
data_CS = pd.read_csv('FPS_counterStrikeGlobalOffensive.csv') 
data_Battlefield4 = pd.read_csv('FPS_battlefield4.csv') 
data_PUBG = pd.read_csv('FPS_playerUnknownsBattlegrounds.csv') 
data_CODWW2 = pd.read_csv('FPS_callOfDutyWW2.csv') 

#printing the data just to check:
print(data_GTAV.head()) 
 

   CpuNumberOfCores  CpuNumberOfThreads  CpuBaseClock  CpuFrequency  CpuTDP  \
0                 8                  16           100          3600      95   
1                 8                   8           100          3600      95   
2                 6                  12           100          3700      95   
3                 4                   8           100          4200      91   
4                 6                   6           100          2900      65   

   CpuTurboClock  GpuBandwidth  GpuBaseClock  GpuBoostClock  GpuMemorySize  \
0           5000        616000          1350           1545          11000   
1           4900        616000          1350           1545          11000   
2           4700        616000          1350           1545          11000   
3           4500        616000          1350           1545          11000   
4           4100        616000          1350           1545          11000   

     FPS  
0  135.2  
1  134.0  
2  123.8  
3  120.2  
4

In [4]:
#Seperating the data in features and label
X_GTAV = data_GTAV.drop(columns = ['FPS', 'CpuBaseClock']).values  
y_GTAV = data_GTAV['FPS'].values 

#Printing into arrays:
print("Features(X):\n", X_GTAV)
print("\nLabel(y):\n", y_GTAV)

Features(X):
 [[    8    16  3600 ...  1350  1545 11000]
 [    8     8  3600 ...  1350  1545 11000]
 [    6    12  3700 ...  1350  1545 11000]
 ...
 [    6    12  3600 ...  1168  1244  4000]
 [    6    12  3400 ...  1168  1244  4000]
 [    8    16  3400 ...  1168  1244  4000]]

Label(y):
 [135.2 134.  123.8 ...  40.3  39.8  38.6]


DOING THE SAME STUFF FOR ALL THE OTHER DATASETS

In [5]:
X_Fortnite = data_Fortnite.drop(columns = ['FPS', 'CpuBaseClock']).values  
y_Fortnite = data_Fortnite['FPS'].values 

X_FarCry5 = data_FarCry5.drop(columns = ['FPS', 'CpuBaseClock']).values  
y_FarCry5 = data_FarCry5['FPS'].values 

X_CS = data_CS.drop(columns = ['FPS', 'CpuBaseClock']).values  
y_CS = data_CS['FPS'].values 

X_Battlefield4 = data_Battlefield4.drop(columns = ['FPS', 'CpuBaseClock']).values  
y_Battlefield4 = data_Battlefield4['FPS'].values 

X_PUBG = data_PUBG.drop(columns = ['FPS', 'CpuBaseClock']).values  
y_PUBG = data_PUBG['FPS'].values 

X_CODWW2 = data_CODWW2.drop(columns = ['FPS', 'CpuBaseClock']).values  
y_CODWW2 = data_CODWW2['FPS'].values 


#Just checking on some random data:
print(X_FarCry5)

[[    8    16  3600 ...  1350  1545 11000]
 [    8     8  3600 ...  1350  1545 11000]
 [    6    12  3700 ...  1350  1545 11000]
 ...
 [    6    12  3600 ...  1168  1244  4000]
 [    6    12  3400 ...  1168  1244  4000]
 [    8    16  3400 ...  1168  1244  4000]]


NORMALISING DATA FOR MODEL'S CONVINIENCE:

We are having 10 features and all of them are of variating ranges so to just get them at a value of 0 to 1:
Taking the newest gen as a reference: i9-14900k paired with RTX 4090 (we didn't consider ryzen 9 7950X3D because it doesn't works on the p and e cores architecture)
No. of cores = 24
No. of threads = 32
CPUbaseClock = Assuming 100(won't affect the data)
CPUfrequency = 5600
CpuTDP = 125
Cpu_TURBO_CLOCK = 6000

GpuBandwidth = 1008*1000 = 1008000
GpuBaseClock = 2235 
GpuBoostClock = 2520
GpuMemorySize = 240000

In [6]:
#Normalising the data:
#Firstly making an the data a np array:

X_GTAV = np.array(X_GTAV, dtype=np.float64)
y_GTAV = np.array(y_GTAV, dtype=np.float64)

for i in range(0, 1026):
    X_GTAV[i][0] = X_GTAV[i][0]/24

for i in range(0, 1026):
    X_GTAV[i][1] = X_GTAV[i][1]/32

for i in range(0, 1026):
    X_GTAV[i][2] = X_GTAV[i][2]/5600

for i in range(0, 1026):
    X_GTAV[i][3] = X_GTAV[i][3]/125

for i in range(0, 1026):
    X_GTAV[i][4] = X_GTAV[i][4]/6000

for i in range(0, 1026):
    X_GTAV[i][5] = X_GTAV[i][5]/1008000

for i in range(0, 1026):
    X_GTAV[i][6] = X_GTAV[i][6]/2235

for i in range(0, 1026):
    X_GTAV[i][7] = X_GTAV[i][7]/2520

for i in range(0, 1026):
    X_GTAV[i][8] = X_GTAV[i][8]/240000

#Normalising the label too:
#For GTAV Avg fps of data: 82.1423 but since it's locked at 185 fps for even best of the systems so we'll use it.
for i in range(0, 1026):
    y_GTAV[i] = y_GTAV[i]/185


# Now checking the data again:
print(X_GTAV[23][6])
print(y_GTAV[5])


0.6778523489932886
0.6372972972972973


NORMALISATION OF DATA OF GTAV SUCCESFULLY DONE

NORMALISATION OF ALL OTHER DATASETS:

callOfDutyWW2

In [7]:

X_CODWW2 = np.array(X_CODWW2, dtype=np.float64)
y_CODWW2 = np.array(y_CODWW2, dtype=np.float64)
for i in range(0, 1026):
    X_CODWW2[i][0] = X_CODWW2[i][0]/24

for i in range(0, 1026):
    X_CODWW2[i][1] = X_CODWW2[i][1]/32

for i in range(0, 1026):
    X_CODWW2[i][2] = X_CODWW2[i][2]/5600

for i in range(0, 1026):
    X_CODWW2[i][3] = X_CODWW2[i][3]/125

for i in range(0, 1026):
    X_CODWW2[i][4] = X_CODWW2[i][4]/6000

for i in range(0, 1026):
    X_CODWW2[i][5] = X_CODWW2[i][5]/1008000

for i in range(0, 1026):
    X_CODWW2[i][6] = X_CODWW2[i][6]/2235

for i in range(0, 1026):
    X_CODWW2[i][7] = X_CODWW2[i][7]/2520

for i in range(0, 1026):
    X_CODWW2[i][8] = X_CODWW2[i][8]/240000

#Normalising the label too:
#For CODWW2 Avg fps of data: 137.197 but since it's locked at 350 fps for even best of the systems so we'll use it.
for i in range(0, 1026):
    y_CODWW2[i] = y_CODWW2[i]/350


# Now checking the data again:
print(X_CODWW2[23][6])
print(y_CODWW2[5])


0.6778523489932886
0.5637142857142857


battlefield4

In [8]:
X_Battlefield4 = np.array(X_Battlefield4, dtype=np.float64)
y_Battlefield4 = np.array(y_Battlefield4, dtype=np.float64)
for i in range(0, 1026):
    X_Battlefield4[i][0] = X_Battlefield4[i][0]/24

for i in range(0, 1026):
    X_Battlefield4[i][1] = X_Battlefield4[i][1]/32
    
for i in range(0, 1026):
    X_Battlefield4[i][2] = X_Battlefield4[i][2]/5600

for i in range(0, 1026):
    X_Battlefield4[i][3] = X_Battlefield4[i][3]/125

for i in range(0, 1026):
    X_Battlefield4[i][4] = X_Battlefield4[i][4]/6000

for i in range(0, 1026):
    X_Battlefield4[i][5] = X_Battlefield4[i][5]/1008000

for i in range(0, 1026):
    X_Battlefield4[i][6] = X_Battlefield4[i][6]/2235

for i in range(0, 1026):
    X_Battlefield4[i][7] = X_Battlefield4[i][7]/2520

for i in range(0, 1026):
    X_Battlefield4[i][8] = X_Battlefield4[i][8]/240000

#Normalising the label too:
#For CODWW2 Avg fps of data: 137.197 but since it's locked at 350 fps for even best of the systems so we'll use it.
for i in range(0, 1026):
    y_Battlefield4[i] = y_Battlefield4[i]/350


# Now checking the data again:
print(X_Battlefield4[23][6])
print(y_Battlefield4[5])

0.6778523489932886
0.25885714285714284


counterStrikeGlobalOffensive

In [9]:
X_CS = np.array(X_CS, dtype=np.float64)
y_CS = np.array(y_CS, dtype=np.float64)
for i in range(0, 1026):
    X_CS[i][0] = X_CS[i][0]/24

for i in range(0, 1026):
    X_CS[i][1] = X_CS[i][1]/32

for i in range(0, 1026):
    X_CS[i][2] = X_CS[i][2]/5600

for i in range(0, 1026):
    X_CS[i][3] = X_CS[i][3]/125

for i in range(0, 1026):
    X_CS[i][4] = X_CS[i][4]/6000

for i in range(0, 1026):
    X_CS[i][5] = X_CS[i][5]/1008000

for i in range(0, 1026):
    X_CS[i][6] = X_CS[i][6]/2235

for i in range(0, 1026):
    X_CS[i][7] = X_CS[i][7]/2520

for i in range(0, 1026):
    X_CS[i][8] = X_CS[i][8]/240000

#Normalising the label too:
#For CS Avg fps of data: 241.017 but since it's locked at 545 fps for even best of the systems so we'll use it.
for i in range(0, 1026):
    y_CS[i] = y_CS[i]/350


# Now checking the data again:
print(X_CS[23][6])
print(y_CS[5])

0.6778523489932886
0.9885714285714285


farCry5

In [10]:
X_FarCry5 = np.array(X_FarCry5, dtype=np.float64)
y_FarCry5 = np.array(y_FarCry5, dtype=np.float64)
for i in range(0, 1026):
    X_FarCry5[i][0] = X_FarCry5[i][0]/24

for i in range(0, 1026):
    X_FarCry5[i][1] = X_FarCry5[i][1]/32
    
for i in range(0, 1026):
    X_FarCry5[i][2] = X_FarCry5[i][2]/5600

for i in range(0, 1026):
    X_FarCry5[i][3] = X_FarCry5[i][3]/125

for i in range(0, 1026):
    X_FarCry5[i][4] = X_FarCry5[i][4]/6000

for i in range(0, 1026):
    X_FarCry5[i][5] = X_FarCry5[i][5]/1008000

for i in range(0, 1026):
    X_FarCry5[i][6] = X_FarCry5[i][6]/2235

for i in range(0, 1026):
    X_FarCry5[i][7] = X_FarCry5[i][7]/2520

for i in range(0, 1026):
    X_FarCry5[i][8] = X_FarCry5[i][8]/240000

#Normalising the label too:
#For FarCry5 Average fps of data: 103.805 but since it's locked at 185 fps for even best of the systems so we'll use it.
for i in range(0, 1026):
    y_FarCry5[i] = y_FarCry5[i]/185


# Now checking the data again:
print(X_FarCry5[23][6])
print(y_FarCry5[5])

0.6778523489932886
0.8064864864864865


fortnite

In [11]:
X_Fortnite = np.array(X_Fortnite, dtype=np.float64)
y_Fortnite = np.array(y_Fortnite, dtype=np.float64)
for i in range(0, 1026):
    X_Fortnite[i][0] = X_Fortnite[i][0]/24

for i in range(0, 1026):
    X_Fortnite[i][1] = X_Fortnite[i][1]/32

for i in range(0, 1026):
    X_Fortnite[i][2] = X_Fortnite[i][2]/5600

for i in range(0, 1026):
    X_Fortnite[i][3] = X_Fortnite[i][3]/125

for i in range(0, 1026):
    X_Fortnite[i][4] = X_Fortnite[i][4]/6000

for i in range(0, 1026):
    X_Fortnite[i][5] = X_Fortnite[i][5]/1008000

for i in range(0, 1026):
    X_Fortnite[i][6] = X_Fortnite[i][6]/2235

for i in range(0, 1026):
    X_Fortnite[i][7] = X_Fortnite[i][7]/2520

for i in range(0, 1026):
    X_Fortnite[i][8] = X_Fortnite[i][8]/240000

#Normalising the label too:
#For Fortnite Average fps of data: 125.466 but since it's locked at 425 fps (for mid to high settings) for even best of the systems so we'll use it.
for i in range(0, 1026):
    y_Fortnite[i] = y_Fortnite[i]/425


# Now checking the data again:
print(X_Fortnite[23][6])
print(y_Fortnite[5])

0.6778523489932886
0.4108235294117647


playerUnknownsBattlegrounds

In [12]:
X_PUBG = np.array(X_PUBG, dtype=np.float64)
y_PUBG = np.array(y_PUBG, dtype=np.float64)
for i in range(0, 1026):
    X_PUBG[i][0] = X_PUBG[i][0]/24

for i in range(0, 1026):
    X_PUBG[i][1] = X_PUBG[i][1]/32

for i in range(0, 1026):
    X_PUBG[i][2] = X_PUBG[i][2]/5600

for i in range(0, 1026):
    X_PUBG[i][3] = X_PUBG[i][3]/125

for i in range(0, 1026):
    X_PUBG[i][4] = X_PUBG[i][4]/6000

for i in range(0, 1026):
    X_PUBG[i][5] = X_PUBG[i][5]/1008000

for i in range(0, 1026):
    X_PUBG[i][6] = X_PUBG[i][6]/2235

for i in range(0, 1026):
    X_PUBG[i][7] = X_PUBG[i][7]/2520

for i in range(0, 1026):
    X_PUBG[i][8] = X_PUBG[i][8]/240000

#Normalising the label too:
#For PUBG Average fps of data: 81.244 but since it's locked at 475 fps (for mid to high settings) for even best of the systems so we'll use it.
for i in range(0, 1026):
    y_PUBG[i] = y_PUBG[i]/475


# Now checking the data again:
print(X_PUBG[23][6])
print(y_PUBG[5])

0.6778523489932886
0.24526315789473685


SPLITING THE DATA IN TRANING/TESTING:

In [13]:
X_GTAV_train, X_GTAV_test, y_GTAV_train, y_GTAV_test = train_test_split(X_GTAV, y_GTAV, test_size = 0.2)

#Doing same with the other data:
X_Fortnite_train, X_Fortnite_test, y_Fortnite_train, y_Fortnite_test = train_test_split(X_Fortnite, y_Fortnite, test_size = 0.2)
X_FarCry5_train, X_FarCry5_test, y_FarCry5_train, y_FarCry5_test = train_test_split(X_FarCry5, y_FarCry5, test_size = 0.2)
X_CS_train, X_CS_test, y_CS_train, y_CS_test = train_test_split(X_CS, y_CS, test_size = 0.2)
X_Battlefield4_train, X_Battlefield4_test, y_Battlefield4_train, y_Battlefield4_test = train_test_split(X_Battlefield4, y_Battlefield4, test_size = 0.2)
X_PUBG_train, X_PUBG_test, y_PUBG_train, y_PUBG_test = train_test_split(X_PUBG, y_PUBG, test_size = 0.2)
X_CODWW2_train, X_CODWW2_test, y_CODWW2_train, y_CODWW2_test = train_test_split(X_CODWW2, y_CODWW2, test_size = 0.2)



In [14]:
#Checking with the dimensions now:
#For Ex. data of GTAV
X_GTAV = np.array(X_GTAV)
print(X_GTAV)
print(y_GTAV)
print("\n", X_GTAV_train)
print(X_GTAV.shape)
print(X_GTAV_train.shape)
print(X_GTAV_train[0])

print(X_GTAV_train.shape)  # Should be (n_samples, n_features)
print(y_GTAV_train.shape)  # Should be (n_samples,)
print(X_GTAV_train.dtype)
print(y_GTAV_train.dtype)



[[0.33333333 0.5        0.64285714 ... 0.60402685 0.61309524 0.04583333]
 [0.33333333 0.25       0.64285714 ... 0.60402685 0.61309524 0.04583333]
 [0.25       0.375      0.66071429 ... 0.60402685 0.61309524 0.04583333]
 ...
 [0.25       0.375      0.64285714 ... 0.52259508 0.49365079 0.01666667]
 [0.25       0.375      0.60714286 ... 0.52259508 0.49365079 0.01666667]
 [0.33333333 0.5        0.60714286 ... 0.52259508 0.49365079 0.01666667]]
[0.73081081 0.72432432 0.66918919 ... 0.21783784 0.21513514 0.20864865]

 [[0.25       0.1875     0.51785714 ... 0.71812081 0.70238095 0.03333333]
 [0.25       0.375      0.66071429 ... 0.65727069 0.61309524 0.03333333]
 [0.25       0.1875     0.5        ... 0.71901566 0.68769841 0.03333333]
 ...
 [0.25       0.375      0.64285714 ... 0.65548098 0.68452381 0.03333333]
 [0.25       0.375      0.64285714 ... 0.55928412 0.53571429 0.06666667]
 [0.33333333 0.5        0.64285714 ... 0.65548098 0.68452381 0.03333333]]
(1026, 9)
(820, 9)
[0.25       0.1875 

PREPROCESSING AND DATA PARTITIONS COMPLETED

                MODEL TRAINING

GTAV

In [15]:

if np.issubdtype(y_GTAV_train.dtype, np.floating):
    discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
    y_GTAV_train = discretizer.fit_transform(y_GTAV_train.reshape(-1, 1)).ravel()


clf_GTAV = MLPClassifier(solver='adam', activation='relu', hidden_layer_sizes=(725, 725))
clf_GTAV.fit(X_GTAV_train, y_GTAV_train)


MLPClassifier(hidden_layer_sizes=(725, 725))

Checking the acc matrix with one of the model:

In [16]:

y_GTAV_test = discretizer.transform(y_GTAV_test.reshape(-1, 1)).ravel()


prediction_GTAV = clf_GTAV.predict(X_GTAV_test)
conf_matrix_GTAV = confusion_matrix(y_GTAV_test, prediction_GTAV)
print("Confusion Matrix:\n", conf_matrix_GTAV)

Confusion Matrix:
 [[ 6 10  0  0  0]
 [ 2 28 15  1  0]
 [ 0 17 22 31  1]
 [ 0  3 27 33  1]
 [ 0  0  1  3  5]]


In [17]:
#CALCULATING ACCURACY:
def accuracy(cm):       
    diagonal = cm.trace()
    sum_of_elements = cm.sum()
    return (diagonal/sum_of_elements) * 100

print(f'The accuracy of our model is: {accuracy(conf_matrix_GTAV)}')  

The accuracy of our model is: 45.63106796116505


BATTLEFIELD_4

In [18]:


if np.issubdtype(y_Battlefield4_train.dtype, np.floating):
    discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
    y_Battlefield4_train = discretizer.fit_transform(y_Battlefield4_train.reshape(-1, 1)).ravel()


clf = MLPClassifier(solver='adam', activation='relu', hidden_layer_sizes=(100, 50, 25))
clf.fit(X_Battlefield4_train, y_Battlefield4_train)


MLPClassifier(hidden_layer_sizes=(100, 50, 25))

In [19]:
y_Battlefield4_test = discretizer.transform(y_Battlefield4_test.reshape(-1, 1)).ravel()


prediction_Battlefield4 = clf.predict(X_Battlefield4_test)
conf_matrix_Battlefield4 = confusion_matrix(y_Battlefield4_test, prediction_Battlefield4)
print("Confusion Matrix:\n", conf_matrix_Battlefield4)

def accuracy(cm):       
    diagonal = cm.trace()
    sum_of_elements = cm.sum()
    return (diagonal/sum_of_elements) * 100

print(f'The accuracy of our model is: {accuracy(conf_matrix_Battlefield4)}') 

Confusion Matrix:
 [[ 0 12  1  0  0]
 [ 0 26 16  1  0]
 [ 0 20 30 30  0]
 [ 0  1 15 41  3]
 [ 0  0  1  4  5]]
The accuracy of our model is: 49.51456310679612


FARCRY_5

In [20]:

if np.issubdtype(y_FarCry5_train.dtype, np.floating):
    discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
    y_FarCry5_train = discretizer.fit_transform(y_FarCry5_train.reshape(-1, 1)).ravel()


clf = MLPClassifier(solver='adam', activation='relu', hidden_layer_sizes=(725, 725))
clf.fit(X_FarCry5_train, y_FarCry5_train)
   

MLPClassifier(hidden_layer_sizes=(725, 725))

In [21]:
y_FarCry5_test = discretizer.transform(y_FarCry5_test.reshape(-1, 1)).ravel()


prediction_FarCry5 = clf.predict(X_FarCry5_test)
conf_matrix_FarCry5 = confusion_matrix(y_FarCry5_test, prediction_FarCry5)
print("Confusion Matrix:\n", conf_matrix_FarCry5)

def accuracy(cm):       
    diagonal = cm.trace()
    sum_of_elements = cm.sum()
    return (diagonal/sum_of_elements) * 100

print(f'The accuracy of our model is: {accuracy(conf_matrix_FarCry5)}')   

Confusion Matrix:
 [[12 12  1  0  0]
 [ 3 33 19  2  0]
 [ 0 20 20 24  0]
 [ 0  4 15 27  1]
 [ 0  0  0  9  4]]
The accuracy of our model is: 46.601941747572816


FORTNITE

In [22]:

if np.issubdtype(y_Fortnite_train.dtype, np.floating):
    discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
    y_Fortnite_train = discretizer.fit_transform(y_Fortnite_train.reshape(-1, 1)).ravel()


clf = MLPClassifier(solver='adam', activation='relu', hidden_layer_sizes=(725, 725))
clf.fit(X_Fortnite_train, y_Fortnite_train)

MLPClassifier(hidden_layer_sizes=(725, 725))

In [23]:
y_Fortnite_test = discretizer.transform(y_Fortnite_test.reshape(-1, 1)).ravel()


prediction_Fortnite = clf.predict(X_Fortnite_test)
conf_matrix_Fortnite = confusion_matrix(y_Fortnite_test, prediction_Fortnite)
print("Confusion Matrix:\n", conf_matrix_Fortnite)

def accuracy(cm):       
    diagonal = cm.trace()
    sum_of_elements = cm.sum()
    return (diagonal/sum_of_elements) * 100

print(f'The accuracy of our model is: {accuracy(conf_matrix_Fortnite)}')   

Confusion Matrix:
 [[ 1 16  2  0  0]
 [ 0 34 10  0  0]
 [ 0 23 42 17  0]
 [ 0  4 24 25  0]
 [ 0  0  1  5  2]]
The accuracy of our model is: 50.48543689320388


CALLOFDUTYWW2

In [24]:
if np.issubdtype(y_Fortnite_train.dtype, np.floating):
    discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
    y_CODWW2_train = discretizer.fit_transform(y_CODWW2_train.reshape(-1, 1)).ravel()


clf = MLPClassifier(solver='adam', activation='relu', hidden_layer_sizes=(725, 725))
clf.fit(X_CODWW2_train, y_CODWW2_train)

MLPClassifier(hidden_layer_sizes=(725, 725))

In [25]:
y_CODWW2_test = discretizer.transform(y_CODWW2_test.reshape(-1, 1)).ravel()


prediction_CODWW2 = clf.predict(X_CODWW2_test)
conf_matrix_CODWW2 = confusion_matrix(y_CODWW2_test, prediction_CODWW2)
print("Confusion Matrix:\n", conf_matrix_CODWW2)

def accuracy(cm):       
    diagonal = cm.trace()
    sum_of_elements = cm.sum()
    return (diagonal/sum_of_elements) * 100

print(f'The accuracy of our model is: {accuracy(conf_matrix_CODWW2)}')   

Confusion Matrix:
 [[ 6  3  2  0  0]
 [ 7 21 23  1  0]
 [ 0 11 44 12  0]
 [ 0  0 29 33  3]
 [ 0  0  0  6  5]]
The accuracy of our model is: 52.9126213592233


CS

In [26]:
if np.issubdtype(y_CS_train.dtype, np.floating):
    discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
    y_CS_train = discretizer.fit_transform(y_CS_train.reshape(-1, 1)).ravel()


clf = MLPClassifier(solver='adam', activation='relu', hidden_layer_sizes=(725, 725))
clf.fit(X_CS_train, y_CS_train)

MLPClassifier(hidden_layer_sizes=(725, 725))

In [27]:
y_CS_test = discretizer.transform(y_CS_test.reshape(-1, 1)).ravel()


prediction_CS = clf.predict(X_CS_test)
conf_matrix_CS = confusion_matrix(y_CS_test, prediction_CS)
print("Confusion Matrix:\n", conf_matrix_CS)

def accuracy(cm):       
    diagonal = cm.trace()
    sum_of_elements = cm.sum()
    return (diagonal/sum_of_elements) * 100

print(f'The accuracy of our model is: {accuracy(conf_matrix_CS)}')   

Confusion Matrix:
 [[ 1 13  0  0  0]
 [ 2 26 20  0  0]
 [ 0 11 33 25  1]
 [ 0  0 29 30  3]
 [ 0  0  1  4  7]]
The accuracy of our model is: 47.0873786407767


PUBG

In [28]:
if np.issubdtype(y_PUBG_train.dtype, np.floating):
    discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
    y_PUBG_train = discretizer.fit_transform(y_PUBG_train.reshape(-1, 1)).ravel()


clf = MLPClassifier(solver='adam', activation='relu', hidden_layer_sizes=(725, 725))
clf.fit(X_PUBG_train, y_PUBG_train)

MLPClassifier(hidden_layer_sizes=(725, 725))

In [29]:
y_PUBG_test = discretizer.transform(y_PUBG_test.reshape(-1, 1)).ravel()


prediction_PUBG = clf.predict(X_PUBG_test)
conf_matrix_PUBG = confusion_matrix(y_PUBG_test, prediction_PUBG)
print("Confusion Matrix:\n", conf_matrix_PUBG)

def accuracy(cm):       
    diagonal = cm.trace()
    sum_of_elements = cm.sum()
    return (diagonal/sum_of_elements) * 100

print(f'The accuracy of our model is: {accuracy(conf_matrix_PUBG)}')   

Confusion Matrix:
 [[ 6  7  3  0  0]
 [10 26 15  0  0]
 [ 1 18 30 20  2]
 [ 0  0 28 18  8]
 [ 0  0  0  3 11]]
The accuracy of our model is: 44.1747572815534


TESTING ON A SPEC OF:
Intel i7-13700HX + Nvidia RTX4070 Super

No. of cores = 16
No. of threads = 24
CPUfrequency = 4500
CpuTDP = 95
Cpu_TURBO_CLOCK = 5000

GpuBandwidth = 1008*1000 = 504000(note: it's a mobile GPU)
GpuBaseClock = 1980
GpuBoostClock = 2575
GpuMemorySize = 120000


In [34]:
#Just an example(just use your own parameters here)
No_of_cores = 16
No_of_threads = 24
CPUfrequency = 4500
CpuTDP = 95
Cpu_TURBO_CLOCK = 5000

GpuBandwidth = 504000
GpuBaseClock = 1980
GpuBoostClock = 2175
GpuMemorySize = 120000


#Normalising features
No_of_cores = No_of_cores /24
No_of_threads = No_of_threads/32
CPUfrequency = CPUfrequency/5600
CpuTDP = CpuTDP/125
Cpu_TURBO_CLOCK = Cpu_TURBO_CLOCK /6000

GpuBandwidth = GpuBandwidth/1008000
GpuBaseClock = GpuBaseClock/2235
GpuBoostClock = GpuBoostClock/2520
GpuMemorySize = GpuMemorySize/240000


featureArr = np.array([[No_of_cores, No_of_threads, CPUfrequency, CpuTDP, Cpu_TURBO_CLOCK, GpuBandwidth, GpuBaseClock,GpuBoostClock, GpuMemorySize]])

fps = clf_GTAV.predict(featureArr)
print(f'FPS is {fps*82.1423}') 

FPS is [328.5692]
